# CREACION DE UN MODELO DE CLASIFICACION DE ORGANICO E INORGANICO

Este proyecto tiene como objetivo entrenar un modelo de clasificación de imágenes utilizando la arquitectura DenseNet121 y el conjunto de datos de imágenes dividido en diferentes categorías

```
['organic', 'plastico', 'vidrio', 'metal', 'carton', 'papel']
```




**Requisitos:**


* Python 3.x
* TensorFlow 2.x
* Numpy
* Matplotlib
* os
* shutil

**Estructura de directorios:**


```
# /imagenes_para_entrenar
    /train
        /organic
        /plastico
        /vidrio
        /metal
        /carton
        /papel
    /test
        /organic
        /plastico
        /vidrio
        /metal
        /carton
        /papel
```



**Permitir utilizar Drive en Google Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import shutil

**Ruta donde se encuentran las imagenes**

In [ ]:
ruta_dataset = '/content/drive/MyDrive/Waste_Classification_Dataset/waste_dataset'
imagenes_organic = os.path.join(ruta_dataset, 'organic')
imagenes_plastico = os.path.join(ruta_dataset, 'recyclable', 'Plastico')
imagenes_vidrio = os.path.join(ruta_dataset, 'recyclable', 'Vidrio')
imagenes_metal = os.path.join(ruta_dataset, 'recyclable', 'Metal')
imagenes_carton = os.path.join(ruta_dataset, 'recyclable', 'Carton')
imagenes_papel = os.path.join(ruta_dataset, 'recyclable', 'Papel')

**Crea los directorios de entrenamiento y prueba**

In [ ]:
train_data_dir = '/content/drive/MyDrive/imagenes_para_entrenar/train'
test_data_dir = '/content/drive/MyDrive/imagenes_para_entrenar/test'

**Copia las imágenes de las categorías en los directorios de entrenamiento y prueba**

In [ ]:
categorias = {
    'organic': imagenes_organic,
    'plastico': imagenes_plastico,
    'vidrio': imagenes_vidrio,
    'metal': imagenes_metal,
    'carton': imagenes_carton,
    'papel': imagenes_papel
}

**Creación de directorios de entrenamiento y prueba para clasificación de imágenes por categoría**

In [ ]:
for categoria, directorio in categorias.items():
    imagenes = os.listdir(directorio)
    num_imagenes = len(imagenes)
    num_imagenes_entrenamiento = int(num_imagenes * 0.8)

    # Copiamos las imágenes para entrenamiento
    train_categoria_dir = os.path.join(train_data_dir, categoria)
    os.makedirs(train_categoria_dir, exist_ok=True)
    for imagen in imagenes[:num_imagenes_entrenamiento]:
        origen = os.path.join(directorio, imagen)
        destino = os.path.join(train_categoria_dir, imagen)
        shutil.copyfile(origen, destino)

    # Copiamos las imágenes para prueba
    test_categoria_dir = os.path.join(test_data_dir, categoria)
    os.makedirs(test_categoria_dir, exist_ok=True)
    for imagen in imagenes[num_imagenes_entrenamiento:]:
        origen = os.path.join(directorio, imagen)
        destino = os.path.join(test_categoria_dir, imagen)
        shutil.copyfile(origen, destino)

**Preprocesamiento de imágenes utilizando ImageDataGenerator de TensorFlow**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Generadores de flujo de datos para entrenamiento y prueba**

In [ ]:
batch_size = 32
num_classes = 6
img_width, img_height = 224, 224

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    classes=['organic', 'plastico', 'vidrio', 'metal', 'carton', 'papel']
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    classes=['organic', 'plastico', 'vidrio', 'metal', 'carton', 'papel']
)

Found 19748 images belonging to 6 classes.
Found 4942 images belonging to 6 classes.


**Definición y entrenamiento del modelo de clasificación**

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Definimos la arquitectura del modelo
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Congelamos las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=8,
    validation_data=test_generator,
    validation_steps=test_generator.n // batch_size
)

Epoch 1/8
617/617 [==============================] - 326s 517ms/step - loss: 0.5242 - accuracy: 0.8234 - val_loss: 0.3909 - val_accuracy: 0.8600
Epoch 2/8
617/617 [==============================] - 316s 512ms/step - loss: 0.3403 - accuracy: 0.8866 - val_loss: 0.3902 - val_accuracy: 0.8634
Epoch 3/8
617/617 [==============================] - 311s 504ms/step - loss: 0.3092 - accuracy: 0.8937 - val_loss: 0.3615 - val_accuracy: 0.8778
Epoch 4/8
617/617 [==============================] - 317s 513ms/step - loss: 0.2936 - accuracy: 0.8994 - val_loss: 0.4407 - val_accuracy: 0.8598
Epoch 5/8
617/617 [==============================] - 313s 507ms/step - loss: 0.2834 - accuracy: 0.9048 - val_loss: 0.3652 - val_accuracy: 0.8778
Epoch 6/8
617/617 [==============================] - 309s 501ms/step - loss: 0.2752 - accuracy: 0.9058 - val_loss: 0.3774 - val_accuracy: 0.8750
Epoch 7/8
617/617 [==============================] - 314s 509ms/step - loss: 0.2704 - accuracy: 0.9064 - val_loss: 0.3818 - val_ac

**Guardar Modelo**

In [ ]:
model.save('/content/drive/MyDrive/modelo_Ing_Requisitos.h5')

In [ ]:
from tensorflow.keras.models import load_model
modelo_guardado = load_model('/content/drive/MyDrive/modelo_Ing_Requisitos.h5')

**Utilizacion del modelo para identificacion**

In [ ]:
from keras.preprocessing import image
import numpy as np

In [ ]:
from PIL import Image

def classify_image(image_path):
    img = Image.open(image_path)
    img = img.resize((img_width, img_height))
    img_array = np.array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    normalized_img_array = expanded_img_array / 255.0

    predictions = model.predict(normalized_img_array)
    predicted_class_index = np.argmax(predictions)
    class_labels = list(train_generator.class_indices.keys())
    predicted_class_name = class_labels[predicted_class_index]

    return predicted_class_name

In [ ]:
imagen_path = '_metal_d5a14eee.jpg' # <======= Introducir imagen
resultado = classify_image(imagen_path)
print("La imagen pertenece a la categoría de:", resultado)

1/1 [==============================] - 0s 31ms/step
La imagen pertenece a la categoría de: metal
